In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from collections import Counter
from IPython.core.display import display, HTML
from sklearn.linear_model import Ridge

In [3]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

df = pd.read_excel('data.xlsx')
df


,avg_ws80m1,sd_ws80m1,med_ws80m1,riq_sp80m1,avg_ws80m2,sd_ws80m2,med_ws80m2,riq_sp80m2,avg_ws30ft,sd_ws30ft,med_ws30ft,riq_sp30ft,avg_wd,med_wd,avg_wg,med_wg,avg_30ft,avg_hum,wind_speed,pow
0,12.290486,2.468048,12.100651,2.366884,12.461335,2.297320,12.557437,3.139359,7.938776,1.463560,8.0,2.00,18.020408,20.0,9.530612,9.0,34.742857,89.040816,5.805,79.5
1,12.108443,1.689733,12.100651,2.458604,12.426631,1.509785,12.557437,0.000000,7.916667,0.961843,8.0,0.00,31.816667,9.0,9.383333,9.0,34.521667,88.983333,5.883,93.8
2,15.628086,2.017991,15.467534,2.366884,15.984571,1.769569,15.696797,1.569680,10.183333,1.127344,10.0,1.00,226.716667,354.5,11.950000,12.0,34.055000,89.816667,6.391,99.5
3,15.211906,1.876468,15.284092,2.366884,16.010733,1.890697,15.696797,3.139359,10.200000,1.204511,10.0,2.00,24.300000,6.5,12.100000,12.0,33.710000,90.266667,6.608,113.8
4,12.258443,1.815307,12.100651,2.000000,12.662083,1.824757,12.557437,3.139359,8.066667,1.162503,8.0,2.00,8.116667,8.0,9.733333,10.0,33.778333,90.116667,6.533,126.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4660,5.060390,3.611628,4.733767,4.733767,4.081167,3.600360,3.139359,3.139359,2.600000,2.293691,2.0,2.00,142.833333,20.5,3.283333,3.0,62.720000,95.283333,5.901,17.7
4661,12.170564,2.224924,12.100651,2.458604,13.525407,2.277281,13.342277,1.962100,8.616667,1.450794,8.5,1.25,323.750000,346.0,10.483333,10.0,62.651667,95.483333,6.717,15.6
4662,16.555196,4.612591,16.100651,7.366884,18.548382,5.189903,18.836156,8.240818,11.816667,3.306346,12.0,5.25,317.733333,312.5,14.900000,15.0,61.900000,95.000000,6.912,97.7
4663,17.014071,4.695941,17.467534,7.458604,19.411705,5.071680,19.620996,9.418078,12.366667,3.231029,12.5,6.00,323.900000,320.0,15.866667,16.5,60.943333,95.000000,7.242,103.7


In [5]:
X = df.iloc[:, 0:19].values
y = df.iloc[:, 19].values.reshape(-1,1)

df.shape, X.shape, type(X)
df.columns
# [avg_ws80m1, sd_ws80m1, med_ws80m1, riq_sp80m1, avg_ws80m2,
#        sd_ws80m2, med_ws80m2, riq_sp80m2, avg_ws30ft, sd_ws30ft,
#        med_ws30ft, riq_sp30ft, avg_wd, med_wd, avg_wg, med_wg,
#        avg_30ft, avg_hum, wind_speed, pow]

Index(['avg_ws80m1', 'sd_ws80m1', 'med_ws80m1', 'riq_sp80m1', 'avg_ws80m2',
       'sd_ws80m2', 'med_ws80m2', 'riq_sp80m2', 'avg_ws30ft', 'sd_ws30ft',
       'med_ws30ft', 'riq_sp30ft', 'avg_wd', 'med_wd', 'avg_wg', 'med_wg',
       'avg_30ft', 'avg_hum', 'wind_speed', 'pow'],
      dtype='object')

In [8]:
from sklearn.model_selection import train_test_split
# X = X[:,cols]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 25)

# Building the Model ------------------------------------------------------------------------

# Fitting regressior to the Training set
from sklearn.linear_model import ElasticNet
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

steps = [
    ('scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('model', ElasticNet(alpha=0.5,l1_ratio=0.6))
]

ElasticNet_pipe = Pipeline(steps)
ElasticNet_pipe.fit(X_train, y_train)
# Predicting the Test set results
y_pred = ElasticNet_pipe.predict(X_test)
# Applying k-Fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = ElasticNet_pipe,scoring='neg_mean_squared_error', X = X_train, y = y_train, cv = 10)
accuracies.mean()



-1778.4349076649091

In [28]:
from sklearn.feature_selection import SequentialFeatureSelector

sfs = SequentialFeatureSelector(direction= 'forward',estimator = ElasticNet_pipe,n_features_to_select=5,scoring='neg_mean_squared_error')
sfs.fit(X_train,y_train)
cols = sfs.get_support()
X_train[:,cols].shape
y_pred = ElasticNet_pipe.predict(X_test)
accuracies = cross_val_score(estimator = ElasticNet_pipe,scoring='neg_mean_squared_error', X = X_train, y = y_train, cv = 10)
accuracies.mean()

-1681.4221894810598

In [17]:
ElasticNet_pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scalar', 'poly', 'model', 'scalar__copy', 'scalar__with_mean', 'scalar__with_std', 'poly__degree', 'poly__include_bias', 'poly__interaction_only', 'poly__order', 'model__alpha', 'model__copy_X', 'model__fit_intercept', 'model__l1_ratio', 'model__max_iter', 'model__normalize', 'model__positive', 'model__precompute', 'model__random_state', 'model__selection', 'model__tol', 'model__warm_start'])

In [33]:

from sklearn.model_selection import GridSearchCV

parameters = [ {'model__alpha': np.logspace(-1, 1,num=3),'model__l1_ratio':np.arange(0.4,0.8,0.1) } ]

from sklearn.metrics import fbeta_score, make_scorer
scoring_func = make_scorer(mean_squared_error)
# grid = dict()
# grid['alpha'] = [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 0.0, 1.0, 10.0, 100.0]
# grid['l1_ratio'] = np.arange(0.3, 1, 0.01)
grid_search = GridSearchCV(estimator = ElasticNet_pipe, 
                           param_grid = parameters,
                           scoring = scoring_func,
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)  



In [23]:
from skopt import BayesSearchCV

parameters = [ {'model__alpha': np.logspace(-3, 1,num=3) ,'model__l'} ]

from sklearn.metrics import fbeta_score, make_scorer
scoring_func = make_scorer(mean_squared_error)
grid_search = BayesSearchCV(estimator = ridge_pipe, 
                           param_grid = parameters,
                           scoring = scoring_func,
                           cv = 10,
                           n_jobs = -1)
grid_search = grid_search.fit(X_train, y_train)

NameError: name 'ridge_pipe' is not defined

In [ ]:
from skopt import BayesSearchCV
# parameter ranges are specified by one of below
from skopt.space import Real, Categorical, Integer
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
X, y = load_iris(True)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                     train_size=0.75,
                                                     random_state=0)
# log-uniform: understand as search over p = exp(x) by varying x
opt = BayesSearchCV(
     SVC(),
     {
         'C': Real(1e-6, 1e+6, prior='log-uniform'),
         'gamma': Real(1e-6, 1e+1, prior='log-uniform'),
         'degree': Integer(1,8),
         'kernel': Categorical(['linear', 'poly', 'rbf']),
     },
     n_iter=32,
     random_state=0
 )
# executes bayesian optimization
_ = opt.fit(X_train, y_train)
# model can be saved, used for predictions or scoring
print(opt.score(X_test, y_test))

In [34]:
res = grid_search.cv_results_
res['mean_test_score']

array([1684.67939209, 1680.65891076, 1677.02617722, 1673.32242181,
       2011.08649918, 1962.01488094, 1908.10200493, 1849.32310085,
       3293.20016319, 3224.58905771, 3136.47789843, 3019.10225217])